# Домашнее задание: декораторы

## Импорт библиотек, установка констант

In [1]:
import requests
import time
import re

from random import randint
from functools import wraps

In [2]:
BOOK_PATH = 'https://www.gutenberg.org/files/2638/2638-0.txt'

## Задание 1

In [3]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло выполнение декорируемой функции, с точностью до тысячной доли секунды
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.perf_counter() 
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        time_dif = end_time - start_time
        print(f'Время выполнения функции {func.__name__} составило: {time_dif:.3f} секунд')
        return result
    return wrapper

## Задание 2

In [4]:
def logging(func):
    """
    Декоратор, который выводит параметры с которыми была вызвана функция
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        params = [arg for arg in args] + [(key, val) for key, val in kwargs.items()]
        print(f'Функция {func.__name__} была вызвана со следующими параметрами: {", ".join(params)}')
        result = func(*args, **kwargs)
        return result

    return wrapper

## Задание 3

In [5]:
def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов декорируемой функции
    """
    cnt_calls = 0
    @wraps(func)
    def wrapper(*args, **kwargs):
        nonlocal cnt_calls 
        cnt_calls += 1
        print(f'Функция {func.__name__} была вызвана: {cnt_calls} раз')
        result = func(*args, **kwargs)
        return result
    return wrapper

## Задание 4

In [6]:
def memo(func):
  """
  Декоратор, запоминающий результаты исполнения функции func, чьи аргументы args должны быть хешируемыми
  """
  cache = {}
  @wraps(func)
  def fmemo(*args):
    nonlocal cache
    cache_key = args
    if cache_key not in cache:
      cache[cache_key] = func(*args)
    return cache[cache_key]
  
  return fmemo

## Тестирование

In [7]:
@counter
@logging
@benchmark
def word_count(word, url=BOOK_PATH):
    """
    Функция для посчета указанного слова на html-странице
    """

    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()

    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз"

print(word_count('whole'))

print(word_count('and'))

print(word_count('first'))

Функция word_count была вызвана: 1 раз
Функция word_count была вызвана со следующими параметрами: whole
Время выполнения функции word_count составило: 1.944 секунд
Cлово whole встречается 176 раз
Функция word_count была вызвана: 2 раз
Функция word_count была вызвана со следующими параметрами: and
Время выполнения функции word_count составило: 1.478 секунд
Cлово and встречается 8747 раз
Функция word_count была вызвана: 3 раз
Функция word_count была вызвана со следующими параметрами: first
Время выполнения функции word_count составило: 1.510 секунд
Cлово first встречается 275 раз


In [8]:
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [9]:
# измеряем время выполнения
start_time = time.perf_counter()
result = fib(30)
end_time = time.perf_counter()
print(f'Результат: {result}, время выполнения: {end_time - start_time:.10f} секунд')

Результат: 832040, время выполнения: 0.2148187080 секунд


In [10]:
@memo
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [11]:
# измеряем время выполнения
start_time = time.perf_counter()
result = fib(30)
end_time = time.perf_counter()
print(f'Результат: {result}, время выполнения: {end_time - start_time:.10f} секунд')

Результат: 832040, время выполнения: 0.0000478750 секунд
